In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Keras
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense

%matplotlib inline

In [2]:
# returns train, inference_encoder and inference_decoder models
def models(n_input, n_output, n_units):
    """
    Gets and prints the spreadsheet's header columns

    Parameters
    ----------
    n_input : str
        Length of input sequence, e.g number of integers
    n_output : bool, optional
        Length of output sequence, e.g number of integers
    n_units : bool, optional
        N umber of cells to create in the encoder and decoder models

    Returns
    -------
    model, encoder_model, decoder_model
        3 models, 
    """
    
    # training encoder
    encoder_inputs = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]

    
    # training decoder
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    # inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)

    # inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

    return model, encoder_model, decoder_model

In [4]:
from random import randint
from numpy import array
from numpy import argmax
from keras.utils import to_categorical

In [7]:
# generate a sequence of random integers
def generate_sequence(length, n_unique):
    return np.random.randint(n_unique, size=length)

In [8]:
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [argmax(vector) for vector in encoded_seq]

In [46]:
# prepare data for the LSTM
def generate_dataset(n_in, cardinality, n_samples):
    X1, X2, y = list(), list(), list()
    for _ in range(n_samples):
        # generate source sequence
        source = generate_sequence(n_in, cardinality)
        # define target sequence
        target = sorted(source)
        # create padded input target sequence
        target_in = [0] + target[:-1]
        # encode
        src_encoded = to_categorical(source, num_classes=cardinality)
        tar_encoded = to_categorical(target, num_classes=cardinality)
        tar2_encoded = to_categorical(target_in, num_classes=cardinality)
        # store
        X1.append(src_encoded)
        X2.append(tar2_encoded)
        y.append(tar_encoded)

    return array(X1), array(X2), array(y)

In [50]:
# Apply encoder-decoder LSTM model

In [44]:
# configure problem
n_features = 50 + 1
n_steps_in = 6
n_steps_out = 6

In [41]:
# define model
train, encoder, decoder = models(n_features, n_features, 128)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [42]:
# generate training dataset
X1, X2, y = generate_dataset(n_steps_in, n_features, 100000)
print(X1.shape, X2.shape, y.shape)
print('X1=%s, X2=%s, y=%s' % (one_hot_decode(X1[0]), one_hot_decode(X2[0]), one_hot_decode(y[0])))

(100000, 6, 51) (100000, 6, 51) (100000, 6, 51)
X1=[47, 30, 24, 4, 15, 30], X2=[0, 4, 15, 24, 30, 30], y=[4, 15, 24, 30, 30, 47]


In [43]:
# train model
train.fit([X1, X2], y, epochs=1)

3125/3125 [==============================] - 30s 10ms/step - loss: 0.3986 - accuracy: 0.9095


In [34]:
# generate target given source sequence
def predict_sequence(encoder, decoder, source, n_steps, cardinality):
    # encode
    state = encoder.predict(source)
    # start of sequence input
    target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
    
    # collect predictions
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h, c = decoder.predict([target_seq] + state)
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        state = [h, c]
        # update target sequence
        target_seq = yhat
    return array(output)

In [45]:
# evaluate LSTM
total, correct = 100, 0
for _ in range(total):
    X1, X2, y = generate_dataset(n_steps_in, n_steps_out, n_features, 1)
    target = predict_sequence(encoder, decoder, X1, n_steps_out, n_features)
    if np.array_equal(one_hot_decode(y[0]), one_hot_decode(target)):
        correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))


TypeError: generate_dataset() takes 3 positional arguments but 4 were given